In [2]:
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski
from rdkit.ML.Descriptors import MoleculeDescriptors

# https://pandas.pydata.org
import pandas as pd

# https://numpy.org/doc/stable/release.html
import numpy as np

#https://github.com/mordred-descriptor/mordred
from mordred import Calculator, descriptors

# pd.set_option('display.max_rows', 1000)
# pd.set_option('display.max_columns',1000)

In [3]:
train = pd.read_csv("./train_II.csv")
test = pd.read_csv("./test_II.csv")
train.shape

(75383, 2)

In [5]:
train.head()

,Id,Expected
0,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O;1644,2
1,CCCCCCCCC(=O)C;2451,2
2,CCCCCCCCCC[N+](C)(C)CCCCCCCCCC.[Cl-];1384,2
3,C1CN(C(=N1)N[N+](=O)[O-])CC2=CN=C(C=C2)Cl;16,2
4,[Na+].[I-];1856,2


In [31]:
def clean_data(data,id_val,is_test):
    carbon_values = []
    flourine_values = []
    oxygen_values = []
    special_char_values = []
    id_values = []
    compound = []
    expected_val = []
    for i in data[id_val]:
        carbon = 0
        flourine = 0
        oxygen = 0
        specialchar = 0
        element = 0
        split_semi = i.split(";")
        compound_split = [*split_semi[0]]
        id_values.append(int(split_semi[-1]))
        compound.append(split_semi[0])
        for i in compound_split:
            if i == "C":
                carbon = carbon+1;
            elif i == "F":
                flourine = flourine+1;
                
            elif i == "O":
                oxygen = oxygen+1;
                
            else:
                specialchar = specialchar+1;
        carbon_values.append(carbon);
        flourine_values.append(flourine)
        oxygen_values.append(oxygen)
        special_char_values.append(specialchar)
    data_frame = {
        "carbon":carbon_values,
        "flourine": flourine_values,
        "oxygen": oxygen_values,
        "specialchar": special_char_values,
        "ID_vals":id_values,
        "compound": compound,
    } 
    data_frame = pd.DataFrame(data_frame)
    return data_frame

In [32]:
dataset = clean_data(train,"Id","N")
dataset["expected"] = train["Expected"]
dataset_test = clean_data(test,"x","Y")
dataset

,carbon,flourine,oxygen,specialchar,ID_vals,compound,expected
0,17,0,2,23,1644,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O,2
1,10,0,1,3,2451,CCCCCCCCC(=O)C,2
2,23,0,0,13,1384,CCCCCCCCCC[N+](C)(C)CCCCCCCCCC.[Cl-],2
3,10,0,2,29,16,C1CN(C(=N1)N[N+](=O)[O-])CC2=CN=C(C=C2)Cl,2
4,0,0,0,10,1856,[Na+].[I-],2
...,...,...,...,...,...,...,...
75378,8,0,4,19,33,COC(=O)NS(=O)(=O)C1=CC=C(C=C1)N,2
75379,10,0,3,21,1632,CCOP(=S)(OCC)OC1=NN(C(=N1)Cl)C(C)C,1
75380,7,0,0,15,1373,C1=CC=C2C(=C1)NC(=S)S2,1
75381,8,0,1,3,2,CCCCC(CC)C=O,2


In [29]:
def RDkit_descriptors(smiles):
    mols = [Chem.MolFromSmiles(i) for i in smiles] 
    calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
    desc_names = calc.GetDescriptorNames()
    
    Mol_descriptors =[]
    for mol in mols:
        calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
        # add hydrogens to molecules
        # Calculate all 200 descriptors for each molecule
#         mol = Chem.MolFromSmiles(mol)
        descriptors = calc.CalcDescriptors(mol)
        Mol_descriptors.append(descriptors)
    return Mol_descriptors,desc_names,mol 

#variancethreshold
#F1score
#XGBoost
#fingerprints, tpsa, logp

In [175]:
Mol_descriptors,desc_names,mol = RDkit_descriptors(dataset["compound"])
Mol_descriptors_test, desc_names_test, mol_test = RDkit_descriptors(dataset_test["compound"])

[23:17:43] Explicit valence for atom # 1 Si, 8, is greater than permitted
[23:17:45] Explicit valence for atom # 1 Si, 8, is greater than permitted
[23:17:46] Explicit valence for atom # 1 Si, 8, is greater than permitted
[23:17:47] Explicit valence for atom # 1 Si, 8, is greater than permitted
[23:17:49] Explicit valence for atom # 1 Si, 8, is greater than permitted
[23:17:50] Explicit valence for atom # 1 Si, 8, is greater than permitted
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/ML/Descriptors/MoleculeDescriptors.py", line 88, in CalcDescriptors
    res[i] = fn(mol)
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/Chem/EState/EState.py", line 76, in MaxEStateIndex
    return max(EStateIndices(mol, force))
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/Chem/EState/EState.py", line 46, in EStateIndices
    nAtoms = mol.GetNumAtoms()
AttributeError: 'NoneType' object has no attribute 'GetNumAtoms'
Traceback (most recent

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/ML/Descriptors/MoleculeDescriptors.py", line 88, in CalcDescriptors
    res[i] = fn(mol)
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/Chem/EState/EState.py", line 76, in MaxEStateIndex
    return max(EStateIndices(mol, force))
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/Chem/EState/EState.py", line 46, in EStateIndices
    nAtoms = mol.GetNumAtoms()
AttributeError: 'NoneType' object has no attribute 'GetNumAtoms'
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/ML/Descriptors/MoleculeDescriptors.py", line 88, in CalcDescriptors
    res[i] = fn(mol)
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/Chem/EState/EState.py", line 83, in MinEStateIndex
    return min(EStateIndices(mol, force))
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/Chem/EState/EState.py", line 46, in EStateIndices
    nAtoms = mol.GetNumAtoms

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/ML/Descriptors/MoleculeDescriptors.py", line 88, in CalcDescriptors
    res[i] = fn(mol)
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/Chem/EState/EState.py", line 76, in MaxEStateIndex
    return max(EStateIndices(mol, force))
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/Chem/EState/EState.py", line 46, in EStateIndices
    nAtoms = mol.GetNumAtoms()
AttributeError: 'NoneType' object has no attribute 'GetNumAtoms'
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/ML/Descriptors/MoleculeDescriptors.py", line 88, in CalcDescriptors
    res[i] = fn(mol)
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/Chem/EState/EState.py", line 83, in MinEStateIndex
    return min(EStateIndices(mol, force))
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/Chem/EState/EState.py", line 46, in EStateIndices
    nAtoms = mol.GetNumAtoms

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/ML/Descriptors/MoleculeDescriptors.py", line 88, in CalcDescriptors
    res[i] = fn(mol)
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/Chem/EState/EState.py", line 76, in MaxEStateIndex
    return max(EStateIndices(mol, force))
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/Chem/EState/EState.py", line 46, in EStateIndices
    nAtoms = mol.GetNumAtoms()
AttributeError: 'NoneType' object has no attribute 'GetNumAtoms'
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/ML/Descriptors/MoleculeDescriptors.py", line 88, in CalcDescriptors
    res[i] = fn(mol)
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/Chem/EState/EState.py", line 83, in MinEStateIndex
    return min(EStateIndices(mol, force))
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/Chem/EState/EState.py", line 46, in EStateIndices
    nAtoms = mol.GetNumAtoms

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/ML/Descriptors/MoleculeDescriptors.py", line 88, in CalcDescriptors
    res[i] = fn(mol)
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/Chem/EState/EState.py", line 76, in MaxEStateIndex
    return max(EStateIndices(mol, force))
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/Chem/EState/EState.py", line 46, in EStateIndices
    nAtoms = mol.GetNumAtoms()
AttributeError: 'NoneType' object has no attribute 'GetNumAtoms'
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/ML/Descriptors/MoleculeDescriptors.py", line 88, in CalcDescriptors
    res[i] = fn(mol)
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/Chem/EState/EState.py", line 83, in MinEStateIndex
    return min(EStateIndices(mol, force))
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/Chem/EState/EState.py", line 46, in EStateIndices
    nAtoms = mol.GetNumAtoms

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/ML/Descriptors/MoleculeDescriptors.py", line 88, in CalcDescriptors
    res[i] = fn(mol)
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/Chem/EState/EState.py", line 76, in MaxEStateIndex
    return max(EStateIndices(mol, force))
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/Chem/EState/EState.py", line 46, in EStateIndices
    nAtoms = mol.GetNumAtoms()
AttributeError: 'NoneType' object has no attribute 'GetNumAtoms'
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/ML/Descriptors/MoleculeDescriptors.py", line 88, in CalcDescriptors
    res[i] = fn(mol)
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/Chem/EState/EState.py", line 83, in MinEStateIndex
    return min(EStateIndices(mol, force))
  File "/opt/anaconda3/lib/python3.9/site-packages/rdkit/Chem/EState/EState.py", line 46, in EStateIndices
    nAtoms = mol.GetNumAtoms

In [183]:
df_with_200_descriptors = pd.DataFrame(Mol_descriptors,columns=desc_names)

df_with_200_descriptors["mol"] = mol

# df_with_200_descriptors = pd.read_csv("./df_with_200_descriptors_csv.csv")
# df_with_200_descriptors["carbon"] = dataset["carbon"]
# df_with_200_descriptors["flourine"] = dataset["flourine"]
# df_with_200_descriptors["oxygen"] = dataset["oxygen"]
# df_with_200_descriptors["specialchar"] = dataset["specialchar"]
df_with_200_descriptors["expected"] = dataset["expected"]
df_with_200_descriptors["ID_vals"] = dataset["ID_vals"]

# df_test = pd.read_csv("./df_test_csv.csv")
df_test = pd.DataFrame(Mol_descriptors_test, columns=desc_names_test)
df_test["ID_vals"] = dataset_test["ID_vals"]
df_test["mol"] = mol_test
# df_test["carbon"] = dataset_test["carbon"]
# df_test["flourine"] = dataset_test["flourine"]
# df_test["oxygen"] = dataset_test["oxygen"]
# df_test["specialchar"] = dataset_test["specialchar"]
# df_test["compound"] = dataset_test["compound"]

df_with_200_descriptors.to_csv("df_with_200_descriptors_csv.csv", index=False)
df_test.to_csv("df_test_csv.csv", index=False)

In [181]:
# df_with_200_descriptors

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,MaxPartialCharge,MinPartialCharge,MaxAbsPartialCharge,MinAbsPartialCharge,FpDensityMorgan1,FpDensityMorgan2,FpDensityMorgan3,BCUT2D_MWHI,BCUT2D_MWLOW,BCUT2D_CHGHI,BCUT2D_CHGLO,BCUT2D_LOGPHI,BCUT2D_LOGPLOW,BCUT2D_MRHI,BCUT2D_MRLOW,BalabanJ,BertzCT,Chi0,Chi0n,Chi0v,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3n,Chi3v,Chi4n,Chi4v,HallKierAlpha,Ipc,Kappa1,Kappa2,Kappa3,LabuteASA,PEOE_VSA1,PEOE_VSA10,PEOE_VSA11,PEOE_VSA12,PEOE_VSA13,PEOE_VSA14,PEOE_VSA2,PEOE_VSA3,PEOE_VSA4,PEOE_VSA5,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,PEOE_VSA9,SMR_VSA1,SMR_VSA10,SMR_VSA2,SMR_VSA3,SMR_VSA4,SMR_VSA5,SMR_VSA6,SMR_VSA7,SMR_VSA8,SMR_VSA9,SlogP_VSA1,SlogP_VSA10,SlogP_VSA11,SlogP_VSA12,SlogP_VSA2,SlogP_VSA3,SlogP_VSA4,SlogP_VSA5,SlogP_VSA6,SlogP_VSA7,SlogP_VSA8,SlogP_VSA9,TPSA,EState_VSA1,EState_VSA10,EState_VSA11,EState_VSA2,EState_VSA3,EState_VSA4,EState_VSA5,EState_VSA6,EState_VSA7,EState_VSA8,EState_VSA9,VSA_EState1,VSA_EState10,VSA_EState2,VSA_EState3,VSA_EState4,VSA_EState5,VSA_EState6,VSA_EState7,VSA_EState8,VSA_EState9,FractionCSP3,HeavyAtomCount,NHOHCount,NOCount,NumAliphaticCarbocycles,NumAliphaticHeterocycles,NumAliphaticRings,NumAromaticCarbocycles,NumAromaticHeterocycles,NumAromaticRings,NumHAcceptors,NumHDonors,NumHeteroatoms,NumRotatableBonds,NumSaturatedCarbocycles,NumSaturatedHeterocycles,NumSaturatedRings,RingCount,MolLogP,MolMR,fr_Al_COO,fr_Al_OH,fr_Al_OH_noTert,fr_ArN,fr_Ar_COO,fr_Ar_N,fr_Ar_NH,fr_Ar_OH,fr_COO,fr_COO2,fr_C_O,fr_C_O_noCOO,fr_C_S,fr_HOCCN,fr_Imine,fr_NH0,fr_NH1,fr_NH2,fr_N_O,fr_Ndealkylation1,fr_Ndealkylation2,fr_Nhpyrrole,fr_SH,fr_aldehyde,fr_alkyl_carbamate,fr_alkyl_halide,fr_allylic_oxid,fr_amide,fr_amidine,fr_aniline,fr_aryl_methyl,fr_azide,fr_azo,fr_barbitur,fr_benzene,fr_benzodiazepine,fr_bicyclic,fr_diazo,fr_dihydropyridine,fr_epoxide,fr_ester,fr_ether,fr_furan,fr_guanido,fr_halogen,fr_hdrzine,fr_hdrzone,fr_imidazole,fr_imide,fr_isocyan,fr_isothiocyan,fr_ketone,fr_ketone_Topliss,fr_lactam,fr_lactone,fr_methoxy,fr_morpholine,fr_nitrile,fr_nitro,fr_nitro_arom,fr_nitro_arom_nonortho,fr_nitroso,fr_oxazole,fr_oxime,fr_para_hydroxylation,fr_phenol,fr_phenol_noOrthoHbond,fr_phos_acid,fr_phos_ester,fr_piperdine,fr_piperzine,fr_priamide,fr_prisulfonamd,fr_pyridine,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,expected,ID_vals
0,9.316200,-1.533785,9.316200,0.150485,0.794714,317.599,306.511,315.982463,100,0,0.200859,-0.507967,0.507967,0.200859,0.684211,1.000000,1.263158,35.582517,9.922283,2.346931,-2.250979,2.455371,-2.178920,6.678262,0.474593,2.474117,498.898914,14.043606,9.724473,11.992260,8.876029,5.522920,6.656814,4.157690,6.526629,2.780170,3.434823,1.903958,2.659887,-1.09,1.538606e+04,14.321871,5.494957,3.470908,125.716648,10.213055,11.499024,0.000000,3.792536,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,59.068288,35.392371,0.000000,5.917906,10.213055,34.802820,0.0,0.000000,0.000000,9.710442,0.000000,59.657840,0.0,11.499024,0.000000,0.000000,11.499024,34.802820,14.005591,0.000000,0.000000,17.044809,48.530937,0.000000,0.000000,0.0,40.46,9.710442,10.213055,0.0,11.499024,0.000000,11.126903,0.000000,48.530937,0.000000,0.000000,34.802820,-1.533785,18.161033,0.000000,18.632400,1.534240,-0.186345,12.975791,0.000000,0.000000,0.000000,0.142857,19,2,2,0,0,0,2,0,2,2,2,5,2,0,0,0,2,4.59990,78.3466,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1644
1,10.532611,0.333788,10.532611,0.333788,0.516641,156.269,136.109,156.151415,66,0,0.129340,-0.300031,0.300031,0.129340,1.000000,1.545455,2.090909,16.137172,10.087958,1.924471,-1.970427,2.060707,-1.884430,5.747933,-0.116631,2.849761,97.045900,8.527098,7.857996,7.857996,5.270056,4.764784,4.764784,3.219782,3.219782,1.882392,1.882392,1.154276,1.1542

In [178]:
# df_test.to_csv("df_test.csv", index=False)

In [156]:
# # from sklearn.ensemble import RandomForestClassifier
# from sklearn.tree import DecisionTreeClassifier
# model = DecisionTreeClassifier()
# X = df_train_final.drop("expected")
# y = df_train_final["expected"]

In [145]:
# len(X)

71144

In [146]:
# len(df_test)

10994

In [157]:
# model.fit(X,y)

DecisionTreeClassifier()

In [158]:
# df_test = df_test.fillna(0)
# predict = model.predict(df_test)
# submission_file = pd.read_csv("./sample_submission.csv")
# submission_file["Predicted"] = predict

# #Submission_file = pd.DataFrame(submission_file)
# submission_file.to_csv("Submission_file_rdkit_2.csv",index=False)

In [159]:
# np.count_nonzero(predict == 2)

9292

In [160]:
# np.count_nonzero(predict == 1)

1702

In [151]:
# df_with_200_descriptors.to_csv("df_with_200_descriptors_csv", index=False)
# df_test.to_csv("df_test_csv", index=False)

# df_train_final

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,MaxPartialCharge,MinPartialCharge,MaxAbsPartialCharge,MinAbsPartialCharge,FpDensityMorgan1,FpDensityMorgan2,FpDensityMorgan3,BCUT2D_MWHI,BCUT2D_MWLOW,BCUT2D_CHGHI,BCUT2D_CHGLO,BCUT2D_LOGPHI,BCUT2D_LOGPLOW,BCUT2D_MRHI,BCUT2D_MRLOW,BalabanJ,BertzCT,Chi0,Chi0n,Chi0v,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3n,Chi3v,Chi4n,Chi4v,HallKierAlpha,Ipc,Kappa1,Kappa2,Kappa3,LabuteASA,PEOE_VSA1,PEOE_VSA10,PEOE_VSA11,PEOE_VSA12,PEOE_VSA13,PEOE_VSA14,PEOE_VSA2,PEOE_VSA3,PEOE_VSA4,PEOE_VSA5,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,PEOE_VSA9,SMR_VSA1,SMR_VSA10,SMR_VSA2,SMR_VSA3,SMR_VSA4,SMR_VSA5,SMR_VSA6,SMR_VSA7,SMR_VSA8,SMR_VSA9,SlogP_VSA1,SlogP_VSA10,SlogP_VSA11,SlogP_VSA12,SlogP_VSA2,SlogP_VSA3,SlogP_VSA4,SlogP_VSA5,SlogP_VSA6,SlogP_VSA7,SlogP_VSA8,SlogP_VSA9,TPSA,EState_VSA1,EState_VSA10,EState_VSA11,EState_VSA2,EState_VSA3,EState_VSA4,EState_VSA5,EState_VSA6,EState_VSA7,EState_VSA8,EState_VSA9,VSA_EState1,VSA_EState10,VSA_EState2,VSA_EState3,VSA_EState4,VSA_EState5,VSA_EState6,VSA_EState7,VSA_EState8,VSA_EState9,FractionCSP3,HeavyAtomCount,NHOHCount,NOCount,NumAliphaticCarbocycles,NumAliphaticHeterocycles,NumAliphaticRings,NumAromaticCarbocycles,NumAromaticHeterocycles,NumAromaticRings,NumHAcceptors,NumHDonors,NumHeteroatoms,NumRotatableBonds,NumSaturatedCarbocycles,NumSaturatedHeterocycles,NumSaturatedRings,RingCount,MolLogP,MolMR,fr_Al_COO,fr_Al_OH,fr_Al_OH_noTert,fr_ArN,fr_Ar_COO,fr_Ar_N,fr_Ar_NH,fr_Ar_OH,fr_COO,fr_COO2,fr_C_O,fr_C_O_noCOO,fr_C_S,fr_HOCCN,fr_Imine,fr_NH0,fr_NH1,fr_NH2,fr_N_O,fr_Ndealkylation1,fr_Ndealkylation2,fr_Nhpyrrole,fr_SH,fr_aldehyde,fr_alkyl_carbamate,fr_alkyl_halide,fr_allylic_oxid,fr_amide,fr_amidine,fr_aniline,fr_aryl_methyl,fr_azide,fr_azo,fr_barbitur,fr_benzene,fr_benzodiazepine,fr_bicyclic,fr_diazo,fr_dihydropyridine,fr_epoxide,fr_ester,fr_ether,fr_furan,fr_guanido,fr_halogen,fr_hdrzine,fr_hdrzone,fr_imidazole,fr_imide,fr_isocyan,fr_isothiocyan,fr_ketone,fr_ketone_Topliss,fr_lactam,fr_lactone,fr_methoxy,fr_morpholine,fr_nitrile,fr_nitro,fr_nitro_arom,fr_nitro_arom_nonortho,fr_nitroso,fr_oxazole,fr_oxime,fr_para_hydroxylation,fr_phenol,fr_phenol_noOrthoHbond,fr_phos_acid,fr_phos_ester,fr_piperdine,fr_piperzine,fr_priamide,fr_prisulfonamd,fr_pyridine,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,expected,ID_vals
0,9.316200,-1.533785,9.316200,0.150485,0.794714,317.599,306.511,315.982463,100,0,0.200859,-0.507967,0.507967,0.200859,0.684211,1.000000,1.263158,35.582517,9.922283,2.346931,-2.250979,2.455371,-2.178920,6.678262,0.474593,2.474117,498.898914,14.043606,9.724473,11.992260,8.876029,5.522920,6.656814,4.157690,6.526629,2.780170,3.434823,1.903958,2.659887,-1.09,1.538606e+04,14.321871,5.494957,3.470908,125.716648,10.213055,11.499024,0.000000,3.792536,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,59.068288,35.392371,0.000000,5.917906,10.213055,34.802820,0.0,0.000000,0.000000,9.710442,0.000000,59.657840,0.0,11.499024,0.000000,0.000000,11.499024,34.802820,14.005591,0.000000,0.000000,17.044809,48.530937,0.000000,0.000000,0.0,40.46,9.710442,10.213055,0.0,11.499024,0.000000,11.126903,0.000000,48.530937,0.000000,0.000000,34.802820,-1.533785,18.161033,0.000000,18.632400,1.534240,-0.186345,12.975791,0.000000,0.000000,0.000000,0.142857,19,2,2,0,0,0,2,0,2,2,2,5,2,0,0,0,2,4.59990,78.3466,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1644
1,10.532611,0.333788,10.532611,0.333788,0.516641,156.269,136.109,156.151415,66,0,0.129340,-0.300031,0.300031,0.129340,1.000000,1.545455,2.090909,16.137172,10.087958,1.924471,-1.970427,2.060707,-1.884430,5.747933,-0.116631,2.849761,97.045900,8.527098,7.857996,7.857996,5.270056,4.764784,4.764784,3.219782,3.219782,1.882392,1.882392,1.154276,1.1542

In [33]:
# def morgan_fpts(data):
#     Morgan_fpts = []
#     for i in data:
#         mol = Chem.MolFromSmiles(i) 
#         fpts =  AllChem.GetMorganFingerprintAsBitVect(mol,2,2048)
#         mfpts = np.array(fpts)
#         Morgan_fpts.append(mfpts)  
#     return np.array(Morgan_fpts)

In [37]:
# # dataset = dataset.loc[dataset["compound"].str.contains("Si"), "compound"] = " "
# # dataset[dataset['compound'].str.contains('Si'), 'compound'] = ''
# dataset.loc[dataset['compound'].str.contains('Si'), 'compound'] = ''
# Morgan_fpts = morgan_fpts(dataset["compound"])
# # Morgan_fpts.shape

In [46]:
# dataset_test.loc[dataset_test['compound'].str.contains('Si'), 'compound'] = ''
# Morgan_fpts_test = morgan_fpts(dataset_test["compound"])

In [39]:
# # dataset['fp'] = dataset["compound"].apply(lambda x: AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(x), 2))
# # test['fp'] = test["Smiles"].apply(lambda x: AllChem.GetMorganFingerprintAsBitVect(x, 2))
# Morgan_fingerprints = pd.DataFrame(Morgan_fpts,columns=['Col_{}'.format(i) for i in range(Morgan_fpts.shape[1])])
# Morgan_fingerprints

,Col_0,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,...,Col_2038,Col_2039,Col_2040,Col_2041,Col_2042,Col_2043,Col_2044,Col_2045,Col_2046,Col_2047
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75378,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
75379,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
75380,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
75381,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
# Morgan_fingerprints_test = pd.DataFrame(Morgan_fpts_test,columns=['Col_{}'.format(i) for i in range(Morgan_fpts_test.shape[1])])
# Morgan_fingerprints_test

,Col_0,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,...,Col_2038,Col_2039,Col_2040,Col_2041,Col_2042,Col_2043,Col_2044,Col_2045,Col_2046,Col_2047
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10989,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10990,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10991,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10992,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
# Morgan_fingerprints.to_csv("Morgan_fingerprints.csv", index=False)
# Morgan_fingerprints_test.to_csv("Morgan_fingerprints_test.csv", index=False)
